In [244]:
# Import libraries
import requests
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import os, os.path as osp
import re

In [550]:
DATA_FOLDER = 'data'
SAMPLE_DATA_FOLDER = DATA_FOLDER + '/htmlSample/'

#SAMPLE_DATA_FOLDER='htmlSample/'
SAMPLE_FILE='data/htmlSample/ff6da2b8d426c56ae77beda595bdcfea.html' #Recipe
SAMPLE_FILE_2='data/htmlSample/ff727d984c9c0048def173c4c97ab52e.html'
SAMPLE_FILE_3='data/htmlSample/ff6f004f00135caa7d6f91b27cbd3a4a.html'
SAMPLE_FILE_4='data/htmlSample/ff70a9eb4e8de4c2f72f281993ab7a55.html'
SAMPLE_FILE_5='data/htmlSample/ff7208a2993418a1b6d50c88b279f229.html'
SAMPLE_FILE_6='data/sample_400/00afe1a62e4d553763db9a9624636141.html'

DATA_FOLDER_EXPANDED = DATA_FOLDER + '/sample_400/'

In [492]:
#remove first letter to avoid capitalization issues?
meat_products = ['meat', 'beef','pork', 'chicken', 'fish', 'lamb','ham', 'turkey','steak','sirloin','veal','hamburger']
units = ['pounds','grams','oz','ounces','kg','kilograms','lbs' ]

In [533]:
# This code is just to test soup on individual html files to verify format---> it is then inserted into analyse_recipe function
with open(SAMPLE_FILE_5) as f:
    page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    ing_wrap_1=soup.find_all('li', class_="ingredient")
    ing_amnts=[]
    ings=[]
    for i in ing_wrap_1:
                ing_amnts.append((i.find('span',class_='value').text+ ' '+i.find('span',class_='type').text))
                ings.append(i.find('span', class_='name').text)
                #print(i)
                #print('-----')
    #print(ing_wrap_1)
    print(ing_amnts)
    print(ings)

[]
[]


In [546]:
#This code is just to test analyse_recipe function on individual html files
with open(SAMPLE_FILE_5) as f:
    page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    
    ingredient_wrapper, is_recipe, ingredients, ingredient_amounts=analyse_recipe(soup,page)
    print(ingredients)

This recipe is from Epicurious
['\n', 'Ingredients', '\n        7-10 Roma tomatoes', None, '4 cups tomato juice (enough to cover all of the tomatoes)', None, '1 package of fresh basil', None, '1 cup heavy whipping cream', None, '½ cup butter', None, 'salt, pepper, and sugar to taste (the sugar balances the acidity)', None, '\n']


In [547]:
#to test with meat ingredients
ingredients.append('4 1/3 lbs beef')
#ingredient_amounts.append('4 1/3 lbs')
print(ingredient_amounts)
print(ingredients)

[]
['\n', 'Ingredients', '\n        7-10 Roma tomatoes', None, '4 cups tomato juice (enough to cover all of the tomatoes)', None, '1 package of fresh basil', None, '1 cup heavy whipping cream', None, '½ cup butter', None, 'salt, pepper, and sugar to taste (the sugar balances the acidity)', None, '\n', '4 1/3 lbs beef']


### Necessary functions:

In [482]:
def convert_to_kg(quant, unit):
    if (unit=='kilogram') or (unit=='kg'):
        amnt_kg=quant
        #print(quant,'kg')
    elif (unit=='pound') or (unit=='lb') or (unit=='lbs') or (unit=='pounds'):
        amnt_kg=quant/2.205
        #print(amnt_kg,'kg')
    elif(unit=='g') or (unit=='gram') or (unit =='grams'):
        amnt_kg=quant/1000
        #print(amnt_kg,'kg')     
    elif(unit=='oz') or (unit=='ounce'):
        amnt_kg=quant/35.274
        #print(amnt_kg, 'kg')
    return(amnt_kg)
    
convert_to_kg(1,'oz')

0.02834949254408346

In [542]:
value = "a string 1/2 with 5 fractions like 2½ in it"
def check_quantity(quant_str):
    #Cleans input string and extracts numerical values
    #Outputs cleaned string, array of numerical values and sum of numerical values
    quant_str=quant_str.replace("½",".5")
    quant_str=quant_str.replace("1/2",".5")
    quant_str=quant_str.replace("1/3", '.33')
    quant_str=quant_str.replace('1/4','.25')
    quant_str=quant_str.replace('3/4','.75')
    quant_vals=re.findall(r"[-+]?\d*\.\d+|\d+", quant_str)
    total_quant=np.sum([float(i) for i in quant_vals])
    return quant_str, quant_vals, total_quant
a,b,c=check_quantity(value)
print('amount ',b)
print('total = ', c)

amount  ['.5', '5', '2.5']
total =  8.0


In [553]:
def extract_amount(ings_in, ing_amnt_in,meat_products_in):
    """
    Inputs: 
    ings_in= list of ingredients (and quantities)
    ing_amnt_in =  list of quantities corresponding to ingredients (empty if quanitites are included in ings_in)
    meat_products_in = list of products that we are searching for
    
    Outputs:
    meat_ingredients_full = list of meat ingredients (full string)
    meat_ingredients_base = list of meat ingredients (from base string meat_products)
    ing_amnt_out = list of corresponding quanities of meat ingredients in kg (=0 if unit not recognized)
    contains_meat = boolean (True if 1+ ingredients are recognized from meat_products list)
    
    """

    meat_ingredients_full = []
    meat_ingredients_base = []
    ing_amnt_out=[]
    contains_meat=False
    
    #Find meat ingredients
    for i in ings_in:
        for meat_product in meat_products_in:
            if i != None:
                #are any of the meat products present in the ingredients? (ignoring capitals with casefold)
                if meat_product in i.casefold(): 
                    contains_meat=True
                    meat_ingredients_full.append(i.casefold()) 
                    meat_ingredients_base.append(meat_product)

    #extract amount from string if amount is not directly available in ing_amnt_in
    if not ing_amnt_in: #if amount is empty 
        for meat_i in meat_ingredients_full:
            meat_i_quant_kg=0
            
            #print('------------')
            #print('Ingredient: ', meat_i) 
            meat_i, quantity_vals, total_quantity=check_quantity(meat_i) #pass string, return cleaned string and total quantity

            
            #find units and convert to kg
            for u in units: 
                if u in meat_i:
            #        print('Quantity in '+ u + ' converted to ')
                    meat_i_quant_kg = convert_to_kg(total_quantity,u)
            ing_amnt_out.append(meat_i_quant_kg)
            #if meat_i_quant_kg==0:
            #    print('Units not recognized for: '+meat_i)
                
                
    else:# if amount is directly available through ing_amnt
        #print('Amount available')
        for meat_i in meat_ingredients_full:
            meat_i_quant_kg=0
            #print('------------')
            #print('Ingredient: ', meat_i)
            
            #get index of ingredient in meat_ingredients_full
            meat_amount=ing_amnt_in[ings_in.index(meat_i)]
            meat_amount, quantity_vals, total_quantity=check_quantity(meat_amount)
            
            #find units and convert to kg
            for u in units: 
                if u in meat_amount.casefold():
                    meat_i_quant_kg = convert_to_kg(total_quantity,u)
                    #print('Quantity in '+ u + ' converted to ', meat_i_quant_kg, 'kg')
            ing_amnt_out.append(meat_i_quant_kg)
            #if meat_i_quant_kg==0:
                #print('Units not recognized for: '+meat_amount+meat_i)
    return meat_ingredients_full, meat_ingredients_base, ing_amnt_out, contains_meat


    
extract_amount(ingredients,ingredient_amounts, meat_products)


([], [], [], False)

In [554]:
def analyse_recipe(soup, page):
    """
    Input:
    soup
    page
    
    Output:
    ing_wrap =  wrapper for ingredients
    is_recipe = boolean (True if website is found in title)
    ings = list of ingredients
    ing_amnts = list of ingredient quantities if directly available through the soup (ingredients and quantities separated)
    """
    ings = []
    ing_amnts = []
    if 'Allrecipes' in soup.title.string:
        print('This recipe is from Allrecipes')
        is_recipe = True
        ing_wrap=soup.find_all(itemprop="recipeIngredient")
        if not ing_wrap:
            print('alternative format used')
            ing_wrap=soup.find_all('li', class_="plaincharacterwrap ingredient")
            for ing in ing_wrap:
                ings.append(ing.text)
            if not ing_wrap:
                print('alternative format 2 needed for Allrecipes')
            #for ing in ing_wrap:
            #    ing_amnts.append(ing.find(itemprop='amount').text)
            #    ings.append(ing.find(itemprop='name').text)
              
    elif 'Epicurious' in soup.title.string:
        print('This recipe is from Epicurious')
        is_recipe=True
        ing_wrap=soup.find('div', id="ingredients")
        for ing in ing_wrap:
            ings.append(ing.string)
    
    elif 'Food Network' in soup.title.string:
        print('This recipe is from FoodNetwork')
        is_recipe=True
        ing_wrap=soup.find_all('li',class_='ingredient')
        for ing in ing_wrap:
            ings.append(ing.text)
        
    elif 'Food.com' in soup.title.string:
        print('This recipe is from Food.com')
        is_recipe=True
        ing_wrap=soup.find_all('li', class_="ingredient")
        for ing in ing_wrap:
            ing_amnts.append((ing.find('span',class_='value').text+ ' '+ing.find('span',class_='type').text))
            ings.append(ing.find('span', class_='name').text)
    
    elif 'Betty Crocker' in soup.title.string:
        print('This recipe is from Betty Crocker')
        is_recipe = True
        ing_wrap=soup.find_all('dl', class_='ingredient')
        for ing in ing_wrap:
            ings.append(ing.getText())
            
    elif 'MyRecipes' in soup.title.string:
        print('This recipe is from MyRecipes')
        is_recipe=True
        ing_wrap=soup.find_all(itemprop="ingredient")
        for ing in ing_wrap:
            ings.append(ing.getText())

    else:
        is_recipe = False
        ing_wrap=None
        
    if not ing_wrap:  #return warning if website is recognized but format/data extraction is not successful
        print('*******')
        print('NEED NEW FORMAT')  
        print('*******')


    return ing_wrap, is_recipe, ings, ing_amnts


In [552]:
#Nadine's loop for all recipes in folder
# data has following row structure
# RecipeName as Identifier - bool contains_meat - list of co2 ingredients - carbonFootprint - ingredients
data=[]
step=0
for filename in os.listdir(DATA_FOLDER_EXPANDED):
    with open(DATA_FOLDER_EXPANDED+filename) as f:
        isTrue=False
        count_exceptions=0
        print('###############################')
        
        # introduce try/catch such that it does no longer stop, when not recoginzeing letter
        try:
            page = f.read()
            soup = BeautifulSoup(page, 'html.parser')
            print('Title: ', soup.title.string)
            print('filename: ', filename)
            ingredient_wrappers, is_recipe, ingredients, ingredient_amounts = analyse_recipe(soup,page)
            print('is recipe: ',is_recipe)
            #print(ingredients)
            #print(ingredient_amounts)
            #different tag for ingredient and different separators in ingredient list between the webpages
            if is_recipe:
                #Extract all ingredients (and ingredient amounts if available)
                #Extract meat ingredients and quantities in kg
                print('ingredient ammount = ', ingredient_amounts)
                meat_ingredients_full, meat_ingredients_base, ingredient_quant,contains_meat=extract_amount(ingredients, ingredient_amounts, meat_products)
                print('does this recipe contain meat? ', contains_meat)
                print('ingredients = ',ingredients)
                print('meat ingredients=', meat_ingredients_base)
                print('ingredient_quantity (kg)= ',ingredient_quant)
                #data.append
                #add row to dataset only if recipe contains meat
                #data.append([soup.title.string, contains_meat, meatlist, carbon_fp(meatlist), ingredients])
            else:
                print(soup.title.string)
        except:
            print('exception')
            count_exceptions=count_exceptions+1
    step=step+1
    if step>=100:
        break
#print(data)
# save data as csv 

###############################
Title:  
	Chicken Breast Cutlets with Artichokes and Capers Recipe - Allrecipes.com

filename:  000a3333ad24828769b6be5a5e1bdb4a.html
This recipe is from Allrecipes
alternative format used
is recipe:  True
ingredient ammount =  []
Units not recognized for: 
                    2 cups chicken broth
does this recipe contain meat?  True
ingredients =  ['\n                    1 cup whole wheat or white flour', '\n                    1/2 teaspoon salt', '\n                    1/8 teaspoon white pepper, or to taste', '\n                    1/8 teaspoon black pepper, or to taste', '\n                    2 pounds chicken breast tenderloins or strips', '\n                    2 tablespoons canola oil', '\n                    2 tablespoons extra-virgin olive oil', '\n                    2 cups chicken broth', '\n                    2 tablespoons fresh lemon juice', '\n                    1 (12 ounce) jar quartered marinated artichoke hearts, with liquid', '\n      

###############################
Title:  
	Baked Asparagus with Balsamic Butter Sauce Recipe - Allrecipes.com

filename:  00a53bf43b5779e20f2ec5bfca29dda4.html
This recipe is from Allrecipes
alternative format used
is recipe:  True
ingredient ammount =  []
does this recipe contain meat?  False
ingredients =  ['\n                    1 bunch fresh asparagus, trimmed', '\n                    cooking spray', '\n                    salt and pepper to taste', '\n                    2 tablespoons butter', '\n                    1 tablespoon soy sauce', '\n                    1 teaspoon balsamic vinegar']
meat ingredients= []
ingredient_quantity (kg)=  []
###############################
exception
###############################
Title:  
	Chicken Corn Chowder Recipe - Allrecipes.com

filename:  00a58140b06973b9ac0da04b4b10f67e.html
This recipe is from Allrecipes
alternative format used
is recipe:  True
ingredient ammount =  []
Units not recognized for: 
                    1 chicken carcass
Unit

Title:  
	Monkfish Provincial Recipe - Allrecipes.com

filename:  00b416deeb3463c68d37a0ae7804fffa.html
This recipe is from Allrecipes
alternative format used
is recipe:  True
ingredient ammount =  []
does this recipe contain meat?  True
ingredients =  ['\n                    salt and pepper to taste', '\n                    Cajun seasoning to taste', '\n                    1/2 cup all-purpose flour', '\n                    1 1/2 pounds monkfish fillets, roughly chopped', '\n                    2 tablespoons olive oil', '\n                    1/4 cup butter', '\n                    3 cloves garlic, minced', '\n                    1 large tomato, diced', '\n                    1 (8 ounce) package sliced fresh mushrooms', '\n                    1/4 cup dry white wine', '\n                    1 tablespoon chopped fresh parsley']
meat ingredients= ['fish']
ingredient_quantity (kg)=  [0.6802721088435374]
###############################
exception
###############################
Title:  Skewe

exception
###############################
exception
###############################
Title:  Cooks.com - Recipes - Steak For Swissing
filename:  00bf8ddb5a5c14ee0c19db5a870a65e5.html
*******
NEED NEW FORMAT
*******
is recipe:  False
Cooks.com - Recipes - Steak For Swissing
###############################
Title:  
	Christmas puddings  - All recipes UK 

filename:  00bfdbaf7df262cd721afa11cbb39283.html
*******
NEED NEW FORMAT
*******
is recipe:  False

	Christmas puddings  - All recipes UK 

###############################
Title:  Twice Baked Potatoes With Buttermilk Recipe - Food.com - 195417
filename:  00c16380158f3fd8b38ebff34cfee581.html
This recipe is from Food.com
exception
###############################
Title:  Santa Fe Chicken Soup Recipe - Food.com - 270992
filename:  00c1927fff4e4fe7190d863118781f58.html
This recipe is from Food.com
exception
###############################
Title:  Halloween Cupcakes - Cupcakes Recipes - Delish.com
filename:  00c27d06b4f5aa662420c6f6b9e26ef9.ht

###############################
Title:  









Black Bean and Sweet Potato Stew with Chilies and Polenta Triangles Recipe Reviews by Epicurious.com Members, Page  1

filename:  00d39f3eb0b38cebdd34b1c3c49f1221.html
This recipe is from Epicurious
exception
###############################
Title:  Wife Saver Breakfast Recipe - Food.com - 13376
filename:  00d3b3165b6883390ab5a84f392eee1c.html
This recipe is from Food.com
exception
###############################
Title:  Hot cocktail recipes: 30+ appetizing drink recipes.
filename:  00d3ed3913a0ee33f89514221443f77a.html
*******
NEED NEW FORMAT
*******
is recipe:  False
Hot cocktail recipes: 30+ appetizing drink recipes.
###############################
Title:  For a Crowd : recipes and cooking : Food Network : Recipes and Cooking : Food Network
filename:  00d40b5797b3d467e1523f6bf2c8f5ac.html
This recipe is from FoodNetwork
*******
NEED NEW FORMAT
*******
is recipe:  True
ingredient ammount =  []
does this recipe contain meat?  False
ingr

###############################
Title:  Strawberry Shortcake Trifle Recipe | MyRecipes.com
filename:  00e0cea8855936712d9d52c21b705128.html
This recipe is from MyRecipes
is recipe:  True
ingredient ammount =  []
does this recipe contain meat?  False
ingredients =  ['\n2 cups\n whole milk\n \n', '\n1/4 teaspoon\n salt\n \n', '\n1/2 cup\n sugar\n \n', '\n1/4 cup\n cornstarch\n \n', '\n2 \n large eggs\n \n', '\n2 tablespoons\n unsalted butter\n \n', '\n1 teaspoon\n vanilla extract\n \n', '\n1 \n loaf pound cake, cut into 16 slices\n \n', '\n1/2 cup\n strawberry jam\n \n', '\n2 cups\n thinly sliced strawberries\n \n', '\n2 tablespoons\n triple sec (or orange juice)\n \n']
meat ingredients= []
ingredient_quantity (kg)=  []


In [449]:
# calculate carbon footprint
#input ingredients
#identify ingredient ammount
#output carbon footprint
def carbon_fp (l):
    """
    takes a list of ingredients contributing to co2 and returns carbon footprint
    """
    c=len(l)
    return c

